In [1]:
badscript = '''contract get_fib_n(n:int){
    let last_fib_calculated : int ;

    entry get_fib(n: int){
        let result: int = fib(n);
        last_fib_calculated = result;
        let a : int = b;
    }

    func fib(n: int) : int{
        let x : int = last_fib_calculated;
        if (n <= 1) {
            return n;
        }
        else {
            
            let a: int = n - 1;
            let b: int = n - 2;
            return fib(a) + fib(b);
        }
    }

    func fib(n: int) : int{
        let x : int = last_fib_calculated;
    }
}'''

In [2]:
script = '''
contract get_fib_n(n:int){
    let last_fib_calculated: int ;

   
    
    func fib(n: int) : int{
      
        else {
            let a: int = n - 1;
            let b: int = n - 2;
            return fib(a) + fib(b);
        }
    
    }
    entry get_fib_n(n: int){
        let result: int = fib(n);
        last_fib_calculated = result;
        return last_fib_calculated;
        
    }
    
}
'''

In [3]:
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast

lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(script))
tokens = process_lexer_tokens(lexer_tokens)

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

0 	 S' -> .<program>, 
	 <program> -> .contractid(<param-list>){<stat_program_list>},  

1 	 S' -> <program>.,  

2 	 <program> -> contract.id(<param-list>){<stat_program_list>},  

3 	 <program> -> contractid.(<param-list>){<stat_program_list>},  

4 	 <program> -> contractid(.<param-list>){<stat_program_list>}, 
	 <param> -> .id:type, 
	 <param-list> -> .<param>,<param-list>, 
	 <param-list> -> .<param>,  

5 	 <program> -> contractid(<param-list>.){<stat_program_list>},  

6 	 <program> -> contractid(<param-list>).{<stat_program_list>},  

7 	 <program> -> contractid(<param-list>){.<stat_program_list>}, 
	 <stat_program_list> -> .<stat_program>, 
	 <stat_program_list> -> .<stat_program_list><stat_program>, 
	 <def-func> -> .funcid(<param-list>):type{<stat_list>}, 
	 <stat_program> -> .<def-entry>, 
	 <stat_program> -> .<storage>, 
	 <stat_program> -> .<def-func>, 
	 <storage> -> .letid:type;, 
	 <def-entry> -> .entryid(<param-list>){<stat_list>},  

8 	 <stat_program> -> <def-func>.

In [4]:
productions, operations = derivation


productions, operations = derivation
ast = build_slr_ast(productions, operations, tokens)
format = FormatVisitor()
print(format.visit(ast))

NameError: name 'FormatVisitor' is not defined

In [ ]:
from visitors.semantic_check_visitor import SemanticCheckerVisitor
semantic_checker = SemanticCheckerVisitor()
semantic_checker.visit(ast)


False
False
False
False
False
False


['In the corpus of the program declare entry functions or variables not this constant 0',
 'Invalid variable b',
 'Function fib is defined after entry point',
 'Invalid variable last_fib_calculated',
 'Function name fib is used',
 'Function fib is defined after entry point',
 'Invalid variable last_fib_calculated']